# Implementing Agentic RAG Using GPT-4o-mini, LlamaIndex, and ChromaDB

This tutorial will guide you through implementing an agentic Retrieval-Augmented Generation (RAG) system using GPT-4o-mini as the Language Model (LLM), LlamaIndex as the LLM data framework, OpenAI for embeddings, and ChromaDB as the vector store.

## Step 1: Environment and Library Setup

First, let's install the necessary libraries:

In [1]:
!pip install --quiet llama-index
!pip install --quiet llama-index-llms-anthropic
!pip install --quiet llama-index-embeddings-openai
!pip install --quiet llama-index-vector-stores-chroma
!pip install --quiet pandas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3

Now, set up the environment variables:


In [2]:
import os
from google.colab import userdata


os.environ["HF_TOKEN"] = userdata.get('Huggingface')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


## Step 2: LLM and Embedding Model Configuration

Configure the LLM and embedding models:

In [3]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# llm = Anthropic(model="claude-3-sonnet-20240229")
llm = OpenAI(model="gpt-4o-mini")

embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    dimensions=256,
    embed_batch_size=20
)

Settings.embed_model = embed_model
Settings.llm = llm

## Step 3: Data Loading and Processing

Load and prepare the Airbnb dataset:

In [4]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("MongoDB/airbnb_embeddings", split="train", streaming=True)
dataset = dataset.take(2000)
dataset_df = pd.DataFrame(dataset)

# Remove pre-existing embeddings
dataset_df = dataset_df.drop(columns=['text_embeddings'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
dataset_df

,_id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,access,...,guests_included,images,host,address,availability,review_scores,reviews,weekly_price,monthly_price,image_embeddings
0,10006546,https://www.airbnb.com/rooms/10006546,Ribeira Charming Duplex,Fantastic duplex apartment with three bedrooms...,Privileged views of the Douro River and Ribeir...,Fantastic duplex apartment with three bedrooms...,"In the neighborhood of the river, you can find...",Lose yourself in the narrow streets and stairc...,Transport: • Metro station and S. Bento railwa...,We are always available to help guests. The ho...,...,6,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '51399391', 'host_url': 'https://w...","{'street': 'Porto, Porto, Portugal', 'suburb':...","{'availability_30': 28, 'availability_60': 47,...","{'review_scores_accuracy': 9, 'review_scores_c...","[{'_id': '58663741', 'date': 2016-01-03 05:00:...",NaN,NaN,"[-0.1302358955, 0.1534578055, 0.0199299306, -0..."
1,10021707,https://www.airbnb.com/rooms/10021707,Private Room in Bushwick,Here exists a very cozy room for rent in a sha...,,Here exists a very cozy room for rent in a sha...,,,,,...,1,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '11275734', 'host_url': 'https://w...","{'street': 'Brooklyn, NY, United States', 'sub...","{'availability_30': 0, 'availability_60': 0, '...","{'review_scores_accuracy': 10, 'review_scores_...","[{'_id': '61050713', 'date': 2016-01-31 05:00:...",NaN,NaN,"[0.0340401195, 0.1742489338, -0.1572628617, 0...."
2,1001265,https://www.airbnb.com/rooms/1001265,Ocean View Waikiki Marina w/prkg,A short distance from Honolulu's billion dolla...,Great studio located on Ala Moana across the s...,A short distance from Honolulu's billion dolla...,You can breath ocean as well as aloha.,,Honolulu does have a very good air conditioned...,"Pool, hot tub and tennis",...,1,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '5448114', 'host_url': 'https://ww...","{'street': 'Honolulu, HI, United States', 'sub...","{'availability_30': 16, 'availability_60': 46,...","{'review_scores_accuracy': 9, 'review_scores_c...","[{'_id': '4765259', 'date': 2013-05-24 04:00:0...",650.0,2150.0,"[-0.1640156209, 0.1256971657, 0.6594450474, -0..."
3,10009999,https://www.airbnb.com/rooms/10009999,Horto flat with small garden,One bedroom + sofa-bed in quiet and bucolic ne...,Lovely one bedroom + sofa-bed in the living ro...,One bedroom + sofa-bed in quiet and bucolic ne...,This charming ground floor flat is located in ...,"There´s a table in the living room now, that d...","Easy access to transport (bus, taxi, car) and ...",,...,1,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '1282196', 'host_url': 'https://ww...","{'street': 'Rio de Janeiro, Rio de Janeiro, Br...","{'availability_30': 0, 'availability_60': 0, '...","{'review_scores_accuracy': None, 'review_score...",[],1492.0,4849.0,"[-0.1292964518, 0.037789464, 0.2443587631, 0.0..."
4,10047964,https://www.airbnb.com/rooms/10047964,Charming Flat in Downtown Moda,Fully furnished 3+1 flat decorated with vintag...,The apartment is composed of 1 big bedroom wit...,Fully furnished 3+1 flat decorated with vintag...,With its diversity Moda- Kadikoy is one of the...,,,,...,1,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '1241644', 'host_url': 'https://ww...","{'street': 'Kadıköy, İstanbul, Turkey', 'subur...","{'availability_30': 27, 'availability_60': 57,...","{'review_scores_accuracy': 10, 'review_scores_...","[{'_id': '68162172', 'date': 2016-04-02 04:00:...",NaN,NaN,"[-0.1006749049, 0.4022984803, -0.1821258366, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,20287455,https://www.airbnb.com/rooms/20287455,Peaceful Refuge in Central Kona,"Enjoy yourself in this bright, airy and clean ...",This one bedroom seperate guest house will giv...,"Enjoy yourself in this bright, airy and clean 

## Step 4: Embedding Generation
Create LlamaIndex documents and generate embeddings:

In [6]:
import json
from llama_index.core import Document
from llama_index.core.schema import MetadataMode
from llama_index.core.node_parser import SentenceSplitter
from tqdm import tqdm

# Create LlamaIndex documents
documents_json = dataset_df.to_json(orient='records')
documents_list = json.loads(documents_json)

In [7]:
llama_documents = []

for document in documents_list:
    for field in ["amenities", "images", "host", "address", "availability", "review_scores", "reviews", "image_embeddings"]:
        document[field] = json.dumps(document[field])

    llama_document = Document(
        text=document["description"],
        metadata=document,
        excluded_llm_metadata_keys=["_id", "transit", "minimum_nights", "maximum_nights", "cancellation_policy", "last_scraped", "calendar_last_scraped", "first_review", "last_review", "security_deposit", "cleaning_fee", "guests_included", "host", "availability", "reviews", "image_embeddings"],
        excluded_embed_metadata_keys=["_id", "transit", "minimum_nights", "maximum_nights", "cancellation_policy", "last_scraped", "calendar_last_scraped", "first_review", "last_review", "security_deposit", "cleaning_fee", "guests_included", "host", "availability", "reviews", "image_embeddings"],
        metadata_template="{key}=>{value}",
        text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
    )
    llama_documents.append(llama_document)



In [8]:
# Observing input examples
print("\nThe LLM sees this: \n", llama_documents[0].get_content(metadata_mode=MetadataMode.LLM))
print("\nThe Embedding model sees this: \n", llama_documents[0].get_content(metadata_mode=MetadataMode.EMBED))


The LLM sees this: 
 Metadata: listing_url=>https://www.airbnb.com/rooms/10006546
name=>Ribeira Charming Duplex
summary=>Fantastic duplex apartment with three bedrooms, located in the historic area of Porto, Ribeira (Cube) - UNESCO World Heritage Site. Centenary building fully rehabilitated, without losing their original character.
space=>Privileged views of the Douro River and Ribeira square, our apartment offers the perfect conditions to discover the history and the charm of Porto. Apartment comfortable, charming, romantic and cozy in the heart of Ribeira. Within walking distance of all the most emblematic places of the city of Porto. The apartment is fully equipped to host 8 people, with cooker, oven, washing machine, dishwasher, microwave, coffee machine (Nespresso) and kettle. The apartment is located in a very typical area of the city that allows to cross with the most picturesque population of the city, welcoming, genuine and happy people that fills the streets with his outspok

In [9]:
# Generate embeddings
base_splitter = SentenceSplitter(chunk_size=5000, chunk_overlap=200)
nodes = base_splitter.get_nodes_from_documents(llama_documents)

pbar = tqdm(total=len(nodes), desc="Embedding Progress", unit="node")

for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode=MetadataMode.EMBED)
    )
    node.embedding = node_embedding
    pbar.update(1)

pbar.close()
print("Embedding process completed!")

Embedding Progress: 100%|██████████| 2000/2000 [13:32<00:00,  2.46node/s]

Embedding process completed!


# Step 5: ChromaDB Setup


In [10]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Create or get a collection
collection_name = "airbnb_listings"
chroma_collection = chroma_client.get_or_create_collection(name=collection_name)

# Create ChromaVectorStore
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

#Step 6: Vector Database Integration
Now, let's add our nodes to the ChromaDB vector store:

In [11]:
from llama_index.core import StorageContext, VectorStoreIndex

# Create a storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index with the storage context
index = VectorStoreIndex(nodes, storage_context=storage_context)

# Optionally, persist the index
index.storage_context.persist()

print(f"Nodes added to ChromaDB collection: {collection_name}")

Nodes added to ChromaDB collection: airbnb_listings


# Step 7: Retriever Tool Creation
Create the retriever tool:

In [12]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine = index.as_query_engine(similarity_top_k=5)

query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="knowledge_base",
        description=(
            "Provides information about Airbnb listings and reviews. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)

In [13]:
query_engine_tool

# Step 8: AI Agent Creation
Create the AI agent:

In [14]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    [query_engine_tool], llm=llm, verbose=True
)
agent = agent_worker.as_agent()

In [15]:
agent

# Step 9: User Interaction
Finally, let's interact with the agent:

In [16]:
response = agent.chat("Tell me the best listing for a place in New York")
print(str(response))

Added user message to memory: Tell me the best listing for a place in New York
=== Calling Function ===
Calling function: knowledge_base with args: {"input": "best Airbnb listing in New York"}
=== Function Output ===
The best Airbnb listing in New York based on the provided information is the "Charming Bedroom in East Village." It features a spacious pre-war apartment with exposed brick, modern appliances, and great light. Located in the heart of the East Village, it offers easy access to vibrant neighborhoods and is just a short walk from Union Square. The listing has received a perfect rating of 100, indicating high satisfaction among guests.
=== LLM Response ===
The best Airbnb listing in New York is the "Charming Bedroom in East Village." This listing features a spacious pre-war apartment with exposed brick, modern appliances, and great natural light. It's located in the heart of the East Village, providing easy access to vibrant neighborhoods and is just a short walk from Union Sq

In [17]:
response = agent.chat("What is the worse one?")
print(str(response))

Added user message to memory: What is the worse one?
=== Calling Function ===
Calling function: knowledge_base with args: {"input": "worst Airbnb listing in New York"}
=== Function Output ===
The listing with the least favorable review scores is the one in Astoria, Queens, which has only one review and lacks detailed ratings. This could indicate a lack of guest experience or feedback, making it less reliable compared to others with more reviews and higher scores.
=== LLM Response ===
The worst Airbnb listing in New York is located in Astoria, Queens. It has only one review and lacks detailed ratings, which may indicate a lack of guest experience or feedback. This makes it less reliable compared to other listings that have more reviews and higher scores.
The worst Airbnb listing in New York is located in Astoria, Queens. It has only one review and lacks detailed ratings, which may indicate a lack of guest experience or feedback. This makes it less reliable compared to other listings tha

In [18]:
response = agent.chat("Can you compare this to one in Miami?")
print(str(response))

Added user message to memory: Can you compare this to one in Miami?
=== Calling Function ===
Calling function: knowledge_base with args: {"input": "best Airbnb listing in Miami"}
=== Function Output ===
There is no information available regarding Airbnb listings in Miami. The provided details focus on various accommodations located in Maui, Hawaii.
=== Calling Function ===
Calling function: knowledge_base with args: {"input": "worst Airbnb listing in Miami"}
=== Function Output ===
I cannot provide information about Airbnb listings in Miami. The available details pertain to listings in New York City. If you have questions about those listings or need assistance with something else, feel free to ask!
=== LLM Response ===
I couldn't find specific information about Airbnb listings in Miami. The available details primarily focus on listings in New York City. If you have any other questions or need assistance with something else, feel free to ask!
I couldn't find specific information about 

In [19]:
response = agent.chat("What other cities are available in the dataset?")
print(str(response))

Added user message to memory: What other cities are available in the dataset?
=== Calling Function ===
Calling function: knowledge_base with args: {"input": "available cities for Airbnb listings in the dataset"}
=== Function Output ===
The available city for Airbnb listings in the dataset is Sydney, Australia.
=== LLM Response ===
The only other city available for Airbnb listings in the dataset is Sydney, Australia. If you have any specific questions about listings there or need further information, feel free to ask!
The only other city available for Airbnb listings in the dataset is Sydney, Australia. If you have any specific questions about listings there or need further information, feel free to ask!
